<a href="https://colab.research.google.com/github/Moonyflay/Diploma_FCA_HSE/blob/main/Comparison_with_other_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
RS1 = 49 # сид

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import warnings

from collections import OrderedDict
from IPython.display import Image, display
from functools import partial
from google.colab import files
from numba import njit, jit

from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import (
    make_scorer, mean_squared_error, max_error,
    root_mean_squared_error, mean_absolute_error
)
from sklearn.model_selection import (
    KFold, GridSearchCV, cross_validate, train_test_split
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor

from xgboost.sklearn import XGBRegressor



In [ ]:
! pip3 install wldhx.yadisk-direct &> /dev/null

In [ ]:
!curl -L $(yadisk-direct https://disk.yandex.ru/d/BErgkyezDcsotg) -o data_2.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 42959  100 42959    0     0  18848      0  0:00:02  0:00:02 --:--:--  133k


In [ ]:
!ls

data_2.xlsx  sample_data


# Datasets

In [ ]:
compounds = pd.read_excel('data_2.xlsx', sheet_name=1)
compounds.drop('Price', axis=1, inplace=True)

cultures = pd.read_excel('data_2.xlsx', sheet_name=2)

ph_ = pd.read_excel('data_2.xlsx', sheet_name=3)

results = pd.read_excel('data_2.xlsx', sheet_name=4)

## Results

results_old - старые данные, results_new - данные, полученные в этом году


In [ ]:
results_old = results.loc[:,['REE', 'Bacteria', 'Is salt', 'Stunting diameter'] ]
results_old['Is salt'] = results['Is salt'].astype(bool)
display(results_old)
print(results_old.shape)

results_new = results.loc[:,['RE, colution r', 'Concentration, mM', 'S.aureus', 'P.aeruginosa',
                             'E.coli', 'K.pneumoniae', 'C.albicans', 'C. parapsilosis', 'C. glabrata'] ]
results_new = results_new[~results_new['RE, colution r'].isna()]
results_rename = {
    'RE, colution r': 'REE',
    'S.aureus': 'Staphylococcus aureus ',
    'P.aeruginosa': 'Pseudomonas aeruginosa ',
    'E.coli': 'Escherichia coli ',
    'K.pneumoniae': 'Klebsiella pneumoniae ',
    'C.albicans': 'Candida albicans',
    'C. parapsilosis': 'Candida parapsilosis',
    'C. glabrata': 'Candida glabrata',
}
results_new.rename(results_rename, axis=1, inplace=True)
results_new['REE'] = results_new['REE'].str.strip()

tmp = pd.DataFrame(columns=['REE', 'Concentration, mM', 'Bacteria','Is salt', 'Stunting diameter'])
tmp['REE'] = results_new.loc[:,'REE']
tmp['Concentration, mM'] = results_new.loc[:,'Concentration, mM']
tmp.fillna({'Bacteria':results_new.columns[2]},  inplace=True)
tmp['Stunting diameter'] = results_new.iloc[:, 2]
for i in range(3,results_new.shape[1]):
    name = results_new.columns[i]
    tmp = pd.concat([tmp, results_new.loc[:,['REE', 'Concentration, mM']]],axis=0, ignore_index=True)
    tmp.fillna({'Bacteria':name},  inplace=True)
    tmp.iloc[(i-2)*results_new.shape[0]: (i-1)*results_new.shape[0], 4] = results_new[name]
results_new = tmp
results_new['Is salt'] = False
results_new

,REE,Bacteria,Is salt,Stunting diameter
0,La,Staphylococcus aureus,False,25
1,Ce,Staphylococcus aureus,False,23
2,Pr,Staphylococcus aureus,False,22
3,Nd,Staphylococcus aureus,False,16
4,Sm,Staphylococcus aureus,False,20
...,...,...,...,...
190,Tm,Candida glabrata,True,52
191,Yb,Candida glabrata,True,41
192,Lu,Candida glabrata,True,55
193,Y,Candida glabrata,True,52


(195, 4)


,REE,"Concentration, mM",Bacteria,Is salt,Stunting diameter
0,Ce,20.0,Staphylococcus aureus,False,0.0
1,Ce,50.0,Staphylococcus aureus,False,12.0
2,Ce,100.0,Staphylococcus aureus,False,20.0
3,Ce,134.0,Staphylococcus aureus,False,22.0
4,Sm,20.0,Staphylococcus aureus,False,0.0
...,...,...,...,...,...
107,Gd,134.0,Candida glabrata,False,0.0
108,Yb,20.0,Candida glabrata,False,0.0
109,Yb,50.0,Candida glabrata,False,0.0
110,Yb,100.0,Candida glabrata,False,0.0


## Compounds

В двух ячейках ниже код для преобразования электронной структуры Electronic structure RE3+ из строки в бинарные признаки  

In [ ]:
levels = ['4s','3d','4p','5s','4d','5p','6s','4f','5d']
nums = [2, 10, 6, 2, 10, 6, 2, 14, 10]
letter_lvl = {'s':'0', 'p':'1', 'd':'2', 'f':'3'}
lvl_letter  = {v: k for k, v in letter_lvl.items()}
full_lvls = OrderedDict(
    [('4s', 2),
     ('3d', 10),
     ('4p', 6),
     ('5s', 2),
     ('4d', 10),
     ('5p', 6),
     ('6s', 2),
     ('4f', 14),
     ('5d', 10)
    ]
)

In [ ]:
def decypher_el_struct(array):
    n = len(levels)
    m = len(array)
    result = {}
    for i in range(n):
        result[levels[i]] = [nums[i] for k in range(m)]

    for a in range(m):
        s = array[a].split('.')
        k = 0
        while levels[k] != s[0][0] + lvl_letter[s[0][1]]:
            k+=1
        i = 0
        while i < len(s):
            result[levels[k]][a] = int(s[i][2:])
            i+=1
            k+=1
        while k < n:
            result[levels[k]][a] = 0
            k+=1
    return result

el_struct = decypher_el_struct(compounds['Electronic structure RE3+'])
el_struct.pop('5d', None) #для 5d все значения одинаковы
el_struct

{'4s': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0],
 '3d': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0],
 '4p': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0],
 '5s': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0],
 '4d': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 0],
 '5p': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0],
 '6s': [0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0],
 '4f': [0, 0, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 0, 0]}

Тип кристаллической структуры Crystal structure type считаем категорией и кодируем целыми числами.

Так как электронная структура для части уровней повторяется (4s, 3d, 4p и 5s, 4d, 5p заполнены одинаково и имеют только два разных значения), сгруппируем эти столбцы в один

In [ ]:
comp = pd.concat([compounds, pd.DataFrame(el_struct)], axis=1)
comp.rename({'Crystal structure type':'CrS'}, inplace=True, axis=1)
comp['CrS'] = comp['CrS'].astype('category').cat.codes
comp.loc[comp['CrS'] == -1, 'CrS'] = np.nan
comp.drop(['Electronic structure RE3+'], axis=1, inplace=True)
for i in range(len(levels[:-3])):
    comp[levels[i]] = comp[levels[i]].apply(lambda x: x == nums[i])

comp.drop(['4s', '3d', '5s', '4d'], axis=1, inplace=True)
comp.rename({'4p': 'Full up tp 4p',
                  '5p': 'Full up to 5p'},
                  axis=1, inplace=True)
comp


,REE,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,CrS,(salt) d1-1H2O,(salt) d1-2H2O,(salt) d1-3H2O,...,R1 (Ln-N),R2 (Ln-O),Coordination number solution 1(Ln-O),Coordination number solution 1(Ln-N),Coordination number solution 2(Ln-O),lgβ,Full up tp 4p,Full up to 5p,6s,4f
0,La,1.03,538.1,1067.0,1850.3,4819.0,0.0,2.527,2.554,2.585,...,NaN,NaN,NaN,NaN,NaN,0.58,True,True,0,0
1,Ce,1.01,534.4,1050.0,1949.0,3547.0,0.0,2.494,2.525,2.529,...,3.23,2.56,9.7,2.6,9.5,0.69,True,True,1,0
2,Pr,0.99,527.0,1020.0,2086.0,3761.0,1.0,2.429,2.447,2.456,...,3.15,2.53,11.0,2.5,10.4,0.69,True,True,2,0
3,Nd,0.98,533.1,1040.0,2130.0,3900.0,1.0,2.429,2.450,2.451,...,3.10,2.51,10.8,2.4,10.8,0.79,True,True,2,1
4,Sm,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,...,3.06,2.48,11.4,1.8,11.1,0.78,True,True,2,3
5,Eu,0.95,547.1,1085.0,2404.0,4120.0,1.0,2.379,2.403,2.406,...,3.05,2.45,11.9,1.6,11.5,0.83,True,True,2,4
6,Gd,0.94,593.4,1170.0,1990.0,4250.0,1.0,2.364,2.389,2.392,...,3.05,2.39,11.5,1.8,13.2,0.47,True,True,2,5
7,Tb,0.92,565.8,1110.0,2114.0,3839.0,1.0,2.350,2.376,2.380,...,3.05,2.39,11.5,2.1,13.2,0.51,True,True,2,6
8,Dy,0.91,573.0,1130.0,2200.0,3990.0,2.0,2.351,2.358,2.401,...,3.03,2.38,11.5,1.4,13.1,0.15,True,True,2,7
9,Ho,0.90,581.0,1140.0,2204.0,4100.0,2.0,2.337,2.341,2.373,...,2.94,2.36,12.0,1.8,13.0,0.25,True,True,2,8


## Cultures

Большая часть признаков в свойствах бактерий линейно зависит друг от друга (например, Gram-stain и Wall thickness), поэтому уберем линейно зависимые столбцы. Каждый из оставшихся признаков будем считать категорией.

In [ ]:
cul = cultures.loc[:, ['Name','Gram-stain', 'Wall structure']]#, 'Well composition']]
#cul.rename({'Well composition':'Wall composition'}, axis=1, inplace=True)
cul['Gram-stain'] = cul['Gram-stain'].astype('category').cat.codes
cul

,Name,Gram-stain,Wall structure
0,Staphylococcus aureus,1,1
1,Escherichia coli,0,2
2,Pseudomonas aeruginosa,0,2
3,Klebsiella pneumoniae,0,2
4,"S,epidermidis",1,1
5,Acinetobacter baumannii,0,2
6,Enterococcus faecalis,1,1
7,Candida albicans,-1,2
8,Candida parapsilosis,-1,2
9,Candida glabrata,-1,2


## pH

Табличку с pH делаем вертикальной для удобства

In [ ]:
ph = ph_.iloc[:4, :].copy()
ph_rename = {
    ' Concentration, mM' : 'Concentration, mM',
    'Ce(NO3)3x6H2O': 'Ce',
    'Sm(NO3)3x6H2O': 'Sm',
    'Gd(NO3)3x6H2O': 'Gd',
    'Yb(NO3)3x5H2O': 'Yb'
}
ph.rename(ph_rename, axis=1, inplace=True)
tmp = pd.DataFrame(columns=['REE', 'pH', 'Concentration, mM'])
tmp['pH'] = np.zeros(shape=16)

tmp.fillna({'REE': ph.columns[1]},  inplace=True)
tmp['pH'] = ph.iloc[:, 1]
tmp['Concentration, mM'] = ph['Concentration, mM']

for i in range(2,ph.shape[1]):
    name = ph.columns[i]
    tmp.iloc[(i-1)*ph.shape[0]: i*ph.shape[0], 0] = name
    tmp.iloc[(i-1)*ph.shape[0]: i*ph.shape[0], 1] = ph[name]
    tmp.iloc[(i-1)*ph.shape[0]: i*ph.shape[0], 2] = ph['Concentration, mM']
ph = tmp
ph

,REE,pH,"Concentration, mM"
0,Ce,5.76,20
1,Ce,5.42,50
2,Ce,5.03,100
3,Ce,4.86,134
4,Sm,5.56,20
5,Sm,5.39,50
6,Sm,5.35,100
7,Sm,5.52,134
8,Gd,5.55,20
9,Gd,5.31,50


## Merging

### DataFrame 1 (old experiments)

Данные старых экспериментов, содержат как растворы, так и соли.

In [ ]:
df_1 = pd.merge(cul, results_old, right_on='Bacteria', left_on='Name')
df_1 = pd.merge(comp, df_1, on='REE')
df_1.drop(['Name', 'Bacteria', 'REE'], axis=1, inplace=True)
df_1 = df_1.sample(frac=1, random_state=RS1).reset_index(drop=True)
df_1.drop_duplicates(keep='first', inplace=True)
df_1

,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,CrS,(salt) d1-1H2O,(salt) d1-2H2O,(salt) d1-3H2O,(salt) d1-4H2O,...,Coordination number solution 2(Ln-O),lgβ,Full up tp 4p,Full up to 5p,6s,4f,Gram-stain,Wall structure,Is salt,Stunting diameter
0,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,11.1,0.78,True,True,2,3,1,1,True,33
1,0.88,596.7,1160.0,2285.0,4120.0,3.0,2.323,2.324,2.353,2.409,...,12.9,0.20,True,True,2,10,0,2,True,30
2,0.87,603.4,1174.8,2417.0,4203.0,4.0,2.301,2.308,2.337,2.388,...,10.6,0.25,True,True,2,11,0,2,False,17
3,0.86,523.5,1340.0,2022.3,4370.0,5.0,2.232,2.270,2.274,NaN,...,11.8,0.56,True,True,2,12,0,2,True,36
4,0.92,565.8,1110.0,2114.0,3839.0,1.0,2.350,2.376,2.380,2.388,...,13.2,0.51,True,True,2,6,0,2,True,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.87,603.4,1174.8,2417.0,4203.0,4.0,2.301,2.308,2.337,2.388,...,10.6,0.25,True,True,2,11,0,2,True,30
190,0.89,589.3,1150.0,2194.0,4120.0,2.0,2.334,2.339,2.365,2.423,...,12.7,0.15,True,True,2,9,1,1,True,48
192,0.90,600.0,1180.0,1980.0,5847.0,1.0,2.333,2.353,2.354,2.368,...,NaN,NaN,True,False,0,0,0,2,True,28
193,0.90,600.0,1180.0,1980.0,5847.0,1.0,2.333,2.353,2.354,2.368,...,NaN,NaN,True,False,0,0,-1,2,True,51


In [ ]:
categorical_1 = np.r_[5:6, 38:40, 42:45]
df_1.columns[categorical_1]

Index(['CrS', 'Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure',
       'Is salt'],
      dtype='object')

In [ ]:
X_1 = df_1.iloc[:,:-1]
y_1 = df_1.iloc[:, -1]

### DataFrame 2 (new experiments)

Данные новых экспериментов, содержат как растворы, так и соли.

In [ ]:
df_2 = pd.merge(ph, results_new, on=['REE', 'Concentration, mM'])
df_2 = pd.merge(cul, df_2 , right_on='Bacteria', left_on='Name', how='right')
df_2 = pd.merge(comp, df_2, on='REE', how='right')
df_2.drop(['Name', 'Bacteria', 'REE', 'Concentration, mM'], axis=1, inplace=True)
df_2 = df_2.sample(frac=1, random_state=RS1).reset_index(drop=True)
df_2.drop_duplicates(keep='first', inplace=True)
df_2

,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,CrS,(salt) d1-1H2O,(salt) d1-2H2O,(salt) d1-3H2O,(salt) d1-4H2O,...,lgβ,Full up tp 4p,Full up to 5p,6s,4f,Gram-stain,Wall structure,pH,Is salt,Stunting diameter
0,1.01,534.4,1050.0,1949.0,3547.0,0.0,2.494,2.525,2.529,2.552,...,0.69,True,True,1,0,0,2,4.86,False,24.0
1,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,-1,2,5.39,False,0.0
2,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,0,2,5.56,False,0.0
3,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,0,2,5.35,False,18.0
4,0.94,593.4,1170.0,1990.0,4250.0,1.0,2.364,2.389,2.392,2.398,...,0.47,True,True,2,5,-1,2,4.91,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.94,593.4,1170.0,1990.0,4250.0,1.0,2.364,2.389,2.392,2.398,...,0.47,True,True,2,5,1,1,5.01,False,12.0
102,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,0,2,5.39,False,14.0
103,1.01,534.4,1050.0,1949.0,3547.0,0.0,2.494,2.525,2.529,2.552,...,0.69,True,True,1,0,-1,2,5.03,False,20.0
104,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,1,1,5.52,False,22.0


In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 111
Data columns (total 47 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Ionic radii                            81 non-null     float64
 1   1st Ionization energy                  81 non-null     float64
 2   2nd Ionization energy                  81 non-null     float64
 3   3rd Ionization energy                  81 non-null     float64
 4   4th Ionization energy                  81 non-null     float64
 5   CrS                                    81 non-null     float64
 6   (salt)  d1-1H2O                        81 non-null     float64
 7   (salt)  d1-2H2O                        81 non-null     float64
 8   (salt)  d1-3H2O                        81 non-null     float64
 9   (salt) d1-4H2O                         81 non-null     float64
 10  (salt)  d1-5H2O                        19 non-null     float64
 11  (salt) d1-1N

In [ ]:
categorical_2 = np.r_[5:6, 38:40, 42:44, 45:46]
df_2.columns[categorical_2]

Index(['CrS', 'Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure',
       'Is salt'],
      dtype='object')

In [ ]:
X_2 = df_2.iloc[:,:-1]
y_2 = df_2.iloc[:, -1]

### DataFrame 3 (combined experiments)

Данные и старых, и новых экспериментов, содержат как растворы, так и соли.

In [ ]:
df_3 = pd.concat([df_2, df_1], axis=0, ignore_index=True)
df_3.drop_duplicates(keep='first', inplace=True)
df_3

,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,CrS,(salt) d1-1H2O,(salt) d1-2H2O,(salt) d1-3H2O,(salt) d1-4H2O,...,lgβ,Full up tp 4p,Full up to 5p,6s,4f,Gram-stain,Wall structure,pH,Is salt,Stunting diameter
0,1.01,534.4,1050.0,1949.0,3547.0,0.0,2.494,2.525,2.529,2.552,...,0.69,True,True,1,0,0,2,4.86,False,24.0
1,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,-1,2,5.39,False,0.0
2,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,0,2,5.56,False,0.0
3,0.96,544.5,1070.0,2260.0,3990.0,1.0,2.395,2.420,2.421,2.427,...,0.78,True,True,2,3,0,2,5.35,False,18.0
4,0.94,593.4,1170.0,1990.0,4250.0,1.0,2.364,2.389,2.392,2.398,...,0.47,True,True,2,5,-1,2,4.91,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,0.87,603.4,1174.8,2417.0,4203.0,4.0,2.301,2.308,2.337,2.388,...,0.25,True,True,2,11,0,2,NaN,True,30.0
257,0.89,589.3,1150.0,2194.0,4120.0,2.0,2.334,2.339,2.365,2.423,...,0.15,True,True,2,9,1,1,NaN,True,48.0
258,0.90,600.0,1180.0,1980.0,5847.0,1.0,2.333,2.353,2.354,2.368,...,NaN,True,False,0,0,0,2,NaN,True,28.0
259,0.90,600.0,1180.0,1980.0,5847.0,1.0,2.333,2.353,2.354,2.368,...,NaN,True,False,0,0,-1,2,NaN,True,51.0


In [ ]:
categorical_3 = np.r_[5:6, 38:40, 42:44, 45 ]
df_3.columns[categorical_3]

Index(['CrS', 'Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure',
       'Is salt'],
      dtype='object')

In [ ]:
X_3 = df_3.iloc[:,:-1]
y_3 = df_3.iloc[:, -1]

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
  display(df_3[X_3.duplicated(keep=False)].sort_values(by=['Ionic radii', 'Gram-stain']))

,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,CrS,(salt) d1-1H2O,(salt) d1-2H2O,(salt) d1-3H2O,(salt) d1-4H2O,(salt) d1-5H2O,(salt) d1-1NO3,(salt) d1-2NO3,(salt) d1-3NO3,(salt) d1-4NO3,(salt) d1-5NO3,(salt) d1-6NO3,(salt) d1,(salt) d2-1NO3,(salt) d2-2NO3,(salt) d2-3NO3,(salt) d3-1H2O,(salt) d3-2H2O,Coordination number (solid) Ln-O,Coordination number (solid) Ln-N,x,n,"Electronegativity, Oganov","Electronegativity, Malliken","Electronegativity, Poland","Electronegativity, Allred-Rochow",R1 (Ln-O),R1 (Ln-N),R2 (Ln-O),Coordination number solution 1(Ln-O),Coordination number solution 1(Ln-N),Coordination number solution 2(Ln-O),lgβ,Full up tp 4p,Full up to 5p,6s,4f,Gram-stain,Wall structure,pH,Is salt,Stunting diameter
118,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,-1,2,NaN,True,41.0
210,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,-1,2,NaN,True,39.0
223,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,-1,2,NaN,True,45.0
134,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,0,2,NaN,True,40.0
154,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,0,2,NaN,False,25.0
170,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,0,2,NaN,False,23.0
231,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,0,2,NaN,True,32.0
123,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,1,1,NaN,True,43.0
252,0.75,633.1,1235.0,2388.6,7090.6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.35,3.340,1.36,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,1,1,NaN,True,46.0
105,0.86,523.5,1340.0,2022.3,4370.0,5.0,2.232,2.270,2.274,NaN,NaN,2.365,2.358,2.378,2.414,2.440,2.474,2.356,2.803,2.827,2.873,4.742,NaN,9.0,3.0,0.0,3.0,2.68,2.890,1.27,1.14,2.30,2.87,2.32,11.5,2.8,11.8,0.56,True,True,2,12,-1,2,NaN,True,48.0


### DataFrame 4 (salt only)

Данные и старых, и новых экспериментов, содержат только соли. Признаки, относящиеся только к растворам, убраны

In [ ]:
df_4 = df_3.drop(['pH'], axis=1)
df_4 = df_4[df_4['Is salt'] == True]
df_4.drop(df_4.columns[31:38], axis=1, inplace=True)
df_4.drop(['Is salt'], axis=1, inplace=True)
df_4 = df_4.sample(frac=1, random_state=RS1).reset_index(drop=True)
df_4

,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,CrS,(salt) d1-1H2O,(salt) d1-2H2O,(salt) d1-3H2O,(salt) d1-4H2O,...,"Electronegativity, Malliken","Electronegativity, Poland","Electronegativity, Allred-Rochow",Full up tp 4p,Full up to 5p,6s,4f,Gram-stain,Wall structure,Stunting diameter
0,0.91,573.0,1130.0,2200.0,3990.0,2.0,2.351,2.358,2.401,2.448,...,3.15,1.22,1.10,True,True,2,7,1,1,48.0
1,0.87,603.4,1174.8,2417.0,4203.0,4.0,2.301,2.308,2.337,2.388,...,3.12,1.10,1.06,True,True,2,11,-1,2,37.0
2,1.03,538.1,1067.0,1850.3,4819.0,0.0,2.527,2.554,2.585,2.560,...,3.06,1.10,1.08,True,True,0,0,0,2,42.0
3,0.98,533.1,1040.0,2130.0,3900.0,1.0,2.429,2.450,2.451,2.463,...,3.72,1.14,1.07,True,True,2,1,-1,2,52.0
4,0.98,533.1,1040.0,2130.0,3900.0,1.0,2.429,2.450,2.451,2.463,...,3.72,1.14,1.07,True,True,2,1,0,2,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0.92,565.8,1110.0,2114.0,3839.0,1.0,2.350,2.376,2.380,2.388,...,3.51,1.10,1.10,True,True,2,6,0,2,38.0
126,1.03,538.1,1067.0,1850.3,4819.0,0.0,2.527,2.554,2.585,2.560,...,3.06,1.10,1.08,True,True,0,0,1,1,45.0
127,0.88,596.7,1160.0,2285.0,4120.0,3.0,2.323,2.324,2.353,2.409,...,3.61,1.25,1.11,True,True,2,10,1,1,44.0
128,0.89,589.3,1150.0,2194.0,4120.0,2.0,2.334,2.339,2.365,2.423,...,3.21,1.24,1.11,True,True,2,9,1,1,48.0


In [ ]:
categorical_4 = np.r_[5:6, 31:33, 35:37]
df_4.columns[categorical_4]

Index(['CrS', 'Full up tp 4p', 'Full up to 5p', 'Gram-stain',
       'Wall structure'],
      dtype='object')

In [ ]:
X_4 = df_4.iloc[:,:-1]
y_4 = df_4.iloc[:, -1]

### DataFrame 5 (solution only)

Данные и старых, и новых экспериментов, содержат как растворы, так и соли. Признаки, относящиеся только к солям, убраны

In [ ]:
df_5 = df_3[df_3['Is salt'] == False].drop('Is salt', axis=1)
df_5.drop(df_5.columns[5:25], axis=1, inplace=True)
df_5.reset_index(drop=True, inplace=True)
df_5

,Ionic radii,1st Ionization energy,2nd Ionization energy,3rd Ionization energy,4th Ionization energy,x,n,"Electronegativity, Oganov","Electronegativity, Malliken","Electronegativity, Poland",...,Coordination number solution 2(Ln-O),lgβ,Full up tp 4p,Full up to 5p,6s,4f,Gram-stain,Wall structure,pH,Stunting diameter
0,1.01,534.4,1050.0,1949.0,3547.0,1.0,5.0,2.61,3.050,1.12,...,9.5,0.69,True,True,1,0,0,2,4.86,24.0
1,0.96,544.5,1070.0,2260.0,3990.0,2.0,4.0,1.90,2.900,1.17,...,11.1,0.78,True,True,2,3,-1,2,5.39,0.0
2,0.96,544.5,1070.0,2260.0,3990.0,2.0,4.0,1.90,2.900,1.17,...,11.1,0.78,True,True,2,3,0,2,5.56,0.0
3,0.96,544.5,1070.0,2260.0,3990.0,2.0,4.0,1.90,2.900,1.17,...,11.1,0.78,True,True,2,3,0,2,5.35,18.0
4,0.94,593.4,1170.0,1990.0,4250.0,2.0,4.0,2.40,3.140,1.20,...,13.2,0.47,True,True,2,5,-1,2,4.91,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,1.03,538.1,1067.0,1850.3,4819.0,1.0,5.0,2.49,3.060,1.10,...,NaN,0.58,True,True,0,0,0,2,NaN,24.0
127,0.91,573.0,1130.0,2200.0,3990.0,1.0,4.0,2.07,3.150,1.22,...,13.1,0.15,True,True,2,7,0,2,NaN,18.0
128,0.86,523.5,1340.0,2022.3,4370.0,0.0,3.0,2.68,2.890,1.27,...,11.8,0.56,True,True,2,12,0,2,NaN,22.0
129,1.00,589.8,1145.4,4912.4,6491.0,NaN,NaN,2.20,3.255,1.00,...,NaN,NaN,False,False,0,0,1,1,NaN,0.0


In [ ]:
categorical_5 = np.r_[18:20, 22:24]
df_5.columns[categorical_5]

Index(['Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure'], dtype='object')

In [ ]:
X_5 = df_5.iloc[:,:-1]
y_5 = df_5.iloc[:, -1]

# Methods testing

### Auxillary

In [ ]:
best_params = [{'XGB': [], 'RFR': [], 'DTR': [], 'kNN': []} for i in range(3)]
best_metrics = [{'XGB': [], 'RFR': [], 'DTR': [], 'kNN': []} for i in range(3)]

In [ ]:
kfold = KFold(n_splits=5, random_state=RS1, shuffle=True)

scoring = {
    'mean_absolute_error': make_scorer(mean_absolute_error),
    'root_mean_squared_error' : make_scorer(root_mean_squared_error),
    'max_error' : make_scorer(max_error)
}

In [ ]:
def count_metrics(results):
    rmse = np.round(np.mean(results['test_root_mean_squared_error']), 4)
    mae = np.round(np.mean(results['test_mean_absolute_error']), 4)
    me = np.round(np.mean(results['test_max_error']), 4)
    return(mae, rmse, me)

def print_results(results):
    mae, rmse, me = count_metrics(results)
    print(f'mae = {mae:0.4f}, root_mse = {rmse:0.4f}, max_error = {me:0.4f}')

In [ ]:
print(X_3.shape, X_4.shape, X_5.shape)

(261, 46) (130, 37) (131, 25)


### Creating nonan datasets

In [ ]:
df_3_nona = df_3.dropna(axis=1)
df_3_nona = df_3_nona.drop_duplicates()
X_3_nona = df_3_nona.iloc[:,:-1]
y_3_nona = df_3_nona.iloc[:, -1]

categorical_3_nona = np.r_[8:10, 12:15]
df_3_nona.columns[categorical_3_nona]

Index(['Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure',
       'Is salt'],
      dtype='object')

In [ ]:
df_4_nona = df_4.dropna(axis=1)
df_4_nona = df_4_nona.drop_duplicates()
X_4_nona = df_4_nona.iloc[:,:-1]
y_4_nona = df_4_nona.iloc[:, -1]

categorical_4_nona = np.r_[8:10, 12:14]
df_4_nona.columns[categorical_4_nona]

Index(['Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure'], dtype='object')

In [ ]:
df_5_nona = df_5.dropna(axis=1)
df_5_nona = df_5_nona.drop_duplicates()
X_5_nona = df_5_nona.iloc[:,:-1]
y_5_nona = df_5_nona.iloc[:, -1]

categorical_5_nona = np.r_[8:10, 12:14]
df_5_nona.columns[categorical_5_nona]

Index(['Full up tp 4p', 'Full up to 5p', 'Gram-stain', 'Wall structure'], dtype='object')

### train test splits

In [ ]:
na_dfs = [(X_3, y_3), (X_4, y_4), (X_5, y_5)]
na_list = []
for x_, y_ in na_dfs:
    na_list.append(train_test_split(x_, y_, random_state=RS1, test_size=0.2))

nona_dfs = [(X_3_nona, y_3_nona), (X_4_nona, y_4_nona), (X_5_nona, y_5_nona)]
nona_list = []
for x_, y_ in nona_dfs:
    nona_list.append(train_test_split(x_, y_, random_state=RS1, test_size=0.2))


### XGBoost

In [ ]:
def xgb(data_list, idx):
    X_train_ = data_list[idx][0]
    X_test_ = data_list[idx][1]
    y_train_ = data_list[idx][2]
    y_test_ = data_list[idx][3]
    parameters = {
        'n_estimators' : [5, 10, 15, 20, 25, 30, 35],
        'max_depth' : range(2, 8, 1),
        'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1,
                            0.125, 0.15, 0.175, 0.2, 0.25]
    }
    model = XGBRegressor()
    best = GridSearchCV(
        estimator=model, param_grid=parameters,
        cv=kfold, verbose=1, refit=True,
        scoring='neg_root_mean_squared_error'
    )
    best.fit(X_train_, y_train_)
    best_params[idx]['XGB'].append(best.best_params_)
    model = XGBRegressor(
        n_estimators=best_params[idx]['XGB'][-1]['n_estimators'],
        max_depth=best_params[idx]['XGB'][-1]['max_depth'],
        learning_rate=best_params[idx]['XGB'][-1]['learning_rate'],
    )
    results = cross_validate(
        estimator=model, X=X_train_, y=y_train_,
        cv=kfold, verbose=3, scoring=scoring
    )

    model = best.best_estimator_
    y_pred = model.predict(X_test_)
    rmse = root_mean_squared_error(y_test_, y_pred)
    mae = mean_absolute_error(y_test_, y_pred)
    me = max_error(y_test_, y_pred)
    metrics = {'mae': mae, 'root_mse': rmse, 'max_error': me}
    best_metrics[idx]['XGB'].append(metrics)

    print(best.best_params_)
    print('\n''Train results')
    print_results(results)
    print('\n''Test results')
    print(f'mae: {mae:0.4f}, root_mse: {rmse:0.4f}, max_error: {me:0.4f}')

#### XGBoost with NaN

In [ ]:
xgb(na_list, 1)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
[CV] END  max_error: (test=10.684) mean_absolute_error: (test=4.379) root_mean_squared_error: (test=5.250) total time=   0.0s
[CV] END  max_error: (test=10.286) mean_absolute_error: (test=4.112) root_mean_squared_error: (test=4.951) total time=   0.0s
[CV] END  max_error: (test=18.582) mean_absolute_error: (test=6.743) root_mean_squared_error: (test=8.431) total time=   0.0s
[CV] END  max_error: (test=14.183) mean_absolute_error: (test=4.557) root_mean_squared_error: (test=6.337) total time=   0.0s
[CV] END  max_error: (test=24.216) mean_absolute_error: (test=5.011) root_mean_squared_error: (test=7.158) total time=   0.0s
{'learning_rate': 0.15, 'max_depth': 2, 'n_estimators': 20}

Train results
mae = 4.9605, root_mse = 6.4252, max_error = 15.5902

Test results
mae: 4.3582, root_mse: 5.1142, max_error: 9.3801


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [ ]:
xgb(na_list, 2)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
[CV] END  max_error: (test=13.192) mean_absolute_error: (test=5.336) root_mean_squared_error: (test=7.035) total time=   0.0s
[CV] END  max_error: (test=14.710) mean_absolute_error: (test=5.410) root_mean_squared_error: (test=6.642) total time=   0.0s
[CV] END  max_error: (test=14.624) mean_absolute_error: (test=5.106) root_mean_squared_error: (test=6.532) total time=   0.0s
[CV] END  max_error: (test=29.948) mean_absolute_error: (test=5.980) root_mean_squared_error: (test=9.411) total time=   0.0s
[CV] END  max_error: (test=34.182) mean_absolute_error: (test=5.941) root_mean_squared_error: (test=9.508) total time=   0.0s
{'learning_rate': 0.175, 'max_depth': 2, 'n_estimators': 10}

Train results
mae = 5.5543, root_mse = 7.8255, max_error = 21.3315

Test results
mae: 5.0851, root_mse: 7.2862, max_error: 21.0622


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished


#### XGBoost without NaN

In [ ]:
xgb(nona_list, 1)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
[CV] END  max_error: (test=10.095) mean_absolute_error: (test=4.381) root_mean_squared_error: (test=5.143) total time=   0.0s
[CV] END  max_error: (test=8.766) mean_absolute_error: (test=3.876) root_mean_squared_error: (test=4.771) total time=   0.0s
[CV] END  max_error: (test=18.080) mean_absolute_error: (test=6.624) root_mean_squared_error: (test=8.223) total time=   0.0s
[CV] END  max_error: (test=13.866) mean_absolute_error: (test=4.639) root_mean_squared_error: (test=6.403) total time=   0.0s
[CV] END  max_error: (test=23.878) mean_absolute_error: (test=5.047) root_mean_squared_error: (test=7.177) total time=   0.0s
{'learning_rate': 0.175, 'max_depth': 2, 'n_estimators': 20}

Train results
mae = 4.9133, root_mse = 6.3435, max_error = 14.9371

Test results
mae: 4.2623, root_mse: 4.9438, max_error: 9.0585


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [ ]:
xgb(nona_list, 2)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
[CV] END  max_error: (test=18.381) mean_absolute_error: (test=4.289) root_mean_squared_error: (test=5.852) total time=   0.0s
[CV] END  max_error: (test=18.778) mean_absolute_error: (test=4.327) root_mean_squared_error: (test=5.988) total time=   0.0s
[CV] END  max_error: (test=28.347) mean_absolute_error: (test=5.396) root_mean_squared_error: (test=8.898) total time=   0.0s
[CV] END  max_error: (test=18.085) mean_absolute_error: (test=3.868) root_mean_squared_error: (test=5.479) total time=   0.0s
[CV] END  max_error: (test=18.716) mean_absolute_error: (test=5.103) root_mean_squared_error: (test=7.394) total time=   0.0s
{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5}

Train results
mae = 4.5965, root_mse = 6.7224, max_error = 20.4613

Test results
mae: 9.2540, root_mse: 11.6406, max_error: 18.9992


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


### Random Forest

In [ ]:
def rfr(data_list, idx):
    X_train_ = data_list[idx][0]
    X_test_ = data_list[idx][1]
    y_train_ = data_list[idx][2]
    y_test_ = data_list[idx][3]
    parameters = {
        'n_estimators' : [5, 10, 15, 20, 25, 30, 35],
        'min_samples_split' : [2, 3, 4, 5, 7, 10, 12, 15],
        'max_depth' : range(2, 15, 1)
    }
    model = RandomForestRegressor()
    best = GridSearchCV(
        estimator=model, param_grid=parameters,
        cv=kfold, verbose=1, refit=True,
        scoring='neg_root_mean_squared_error'
    )
    best.fit(X_train_, y_train_)

    best_params[idx]['RFR'].append(best.best_params_)
    model = RandomForestRegressor(
        n_estimators=best_params[idx]['RFR'][-1]['n_estimators'],
        max_depth=best_params[idx]['RFR'][-1]['max_depth'],
        min_samples_split=best_params[idx]['RFR'][-1]['min_samples_split']
    )
    results = cross_validate(
        estimator=model, X=X_train_, y=y_train_,
        cv=kfold,  verbose=4, scoring=scoring
    )

    model = best.best_estimator_
    y_pred = model.predict(X_test_)
    rmse = root_mean_squared_error(y_test_, y_pred)
    mae = mean_absolute_error(y_test_, y_pred)
    me = max_error(y_test_, y_pred)
    metrics = {'mae': mae, 'root_mse': rmse, 'max_error': me}
    best_metrics[idx]['RFR'].append(metrics)

    print(best.best_params_)
    print('Train results')
    print_results(results)
    print()
    print('Test results')
    print(f'mae: {mae:0.4f}, root_mse: {rmse:0.4f}, max_error: {me:0.4f}')

#### Random Forest with NaN

In [ ]:
rfr(na_list, 1)

Fitting 5 folds for each of 728 candidates, totalling 3640 fits
[CV] END  max_error: (test=10.915) mean_absolute_error: (test=4.124) root_mean_squared_error: (test=5.052) total time=   0.0s
[CV] END  max_error: (test=11.811) mean_absolute_error: (test=4.366) root_mean_squared_error: (test=5.344) total time=   0.0s
[CV] END  max_error: (test=18.285) mean_absolute_error: (test=6.521) root_mean_squared_error: (test=8.208) total time=   0.0s
[CV] END  max_error: (test=15.766) mean_absolute_error: (test=5.525) root_mean_squared_error: (test=7.346) total time=   0.0s
[CV] END  max_error: (test=29.829) mean_absolute_error: (test=5.247) root_mean_squared_error: (test=8.089) total time=   0.0s
{'max_depth': 3, 'min_samples_split': 15, 'n_estimators': 20}
Train results
mae = 5.1566, root_mse = 6.8079, max_error = 17.3211

Test results
mae: 4.2735, root_mse: 5.1274, max_error: 9.0301


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [ ]:
rfr(na_list, 2)

Fitting 5 folds for each of 336 candidates, totalling 1680 fits
[CV] END  max_error: (test=16.893) mean_absolute_error: (test=5.537) root_mean_squared_error: (test=7.921) total time=   0.0s
[CV] END  max_error: (test=19.447) mean_absolute_error: (test=5.081) root_mean_squared_error: (test=6.959) total time=   0.0s
[CV] END  max_error: (test=19.043) mean_absolute_error: (test=5.849) root_mean_squared_error: (test=7.598) total time=   0.0s
[CV] END  max_error: (test=46.000) mean_absolute_error: (test=6.241) root_mean_squared_error: (test=11.843) total time=   0.0s
[CV] END  max_error: (test=36.490) mean_absolute_error: (test=5.712) root_mean_squared_error: (test=9.884) total time=   0.0s
{'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 5}
Train results
mae = 5.6840, root_mse = 8.8411, max_error = 27.5747

Test results
mae: 4.5339, root_mse: 7.4537, max_error: 28.3500


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


####Random Forest without NaN

In [ ]:
rfr(nona_list, 1)

Fitting 5 folds for each of 728 candidates, totalling 3640 fits
[CV] END  max_error: (test=10.383) mean_absolute_error: (test=4.134) root_mean_squared_error: (test=4.910) total time=   0.0s
[CV] END  max_error: (test=9.978) mean_absolute_error: (test=4.051) root_mean_squared_error: (test=5.242) total time=   0.0s
[CV] END  max_error: (test=15.362) mean_absolute_error: (test=6.332) root_mean_squared_error: (test=7.579) total time=   0.0s
[CV] END  max_error: (test=17.980) mean_absolute_error: (test=5.688) root_mean_squared_error: (test=7.570) total time=   0.0s
[CV] END  max_error: (test=27.061) mean_absolute_error: (test=5.851) root_mean_squared_error: (test=8.038) total time=   0.0s
{'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 10}
Train results
mae = 5.2113, root_mse = 6.6675, max_error = 16.1529

Test results
mae: 4.2090, root_mse: 5.0381, max_error: 9.7428


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [ ]:
rfr(nona_list, 2)

Fitting 5 folds for each of 336 candidates, totalling 1680 fits
[CV] END  max_error: (test=19.590) mean_absolute_error: (test=3.736) root_mean_squared_error: (test=5.684) total time=   0.0s
[CV] END  max_error: (test=31.413) mean_absolute_error: (test=5.676) root_mean_squared_error: (test=9.455) total time=   0.0s
[CV] END  max_error: (test=40.122) mean_absolute_error: (test=6.056) root_mean_squared_error: (test=11.385) total time=   0.0s
[CV] END  max_error: (test=16.662) mean_absolute_error: (test=4.291) root_mean_squared_error: (test=5.649) total time=   0.0s
[CV] END  max_error: (test=17.891) mean_absolute_error: (test=5.431) root_mean_squared_error: (test=7.516) total time=   0.0s
{'max_depth': 6, 'min_samples_split': 12, 'n_estimators': 5}
Train results
mae = 5.0379, root_mse = 7.9376, max_error = 25.1357

Test results
mae: 10.3458, root_mse: 13.3240, max_error: 36.2500


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


### KNearestNeighbors

In [ ]:
def knn(data_list, categ_list, idx):
    X_train_ = data_list[idx][0]
    X_test_ = data_list[idx][1]
    y_train_ = data_list[idx][2]
    y_test_ = data_list[idx][3]

    categ_cols = list(X_train_.columns[categ_list[idx]])
    numeric_cols = [c for c in X_train_.columns if c not in categ_cols]

    parameters = {
        'knn__n_neighbors' : np.arange(2, 36),
        'knn__weights' : ['uniform', 'distance'],
        'knn__p': [1, 2]
    }
    model = Pipeline([
        ('transformer', make_column_transformer(
            (OneHotEncoder(sparse_output=False), categ_cols),
            (MinMaxScaler(), numeric_cols),
            remainder='passthrough')
        ),
        ('knn', KNeighborsRegressor())]
    )
    best = GridSearchCV(
        estimator=model, param_grid=parameters,
        cv=kfold, verbose=1, refit=True,
        scoring='neg_root_mean_squared_error'
    )
    best.fit(X_train_, y_train_)

    best_params[idx]['kNN'].append(best.best_params_)
    model = Pipeline([
        ('transformer', make_column_transformer(
            (OneHotEncoder(sparse_output=False), categ_cols),
            (MinMaxScaler(), numeric_cols),
            remainder='passthrough')
        ),
        ('knn', KNeighborsRegressor(
            p=best_params[idx]['kNN'][-1]['knn__p'],
            n_neighbors=best_params[idx]['kNN'][-1]['knn__n_neighbors'],
            weights=best_params[idx]['kNN'][-1]['knn__weights']))]
    )
    results = cross_validate(
        estimator=model, X=X_train_, y=y_train_,
        cv=kfold, verbose=3, scoring=scoring
    )

    model = best.best_estimator_
    y_pred = model.predict(X_test_)
    rmse = root_mean_squared_error(y_test_, y_pred)
    mae = mean_absolute_error(y_test_, y_pred)
    me = max_error(y_test_, y_pred)
    metrics = {'mae': mae, 'root_mse': rmse, 'max_error': me}
    best_metrics[idx]['kNN'].append(metrics)

    print(best.best_params_)
    print('Train results')
    print_results(results)
    print()
    print('Test results')
    print(f'mae: {mae:0.4f}, root_mse: {rmse:0.4f}, max_error: {me:0.4f}')

#### Decision Tree without NaN

In [ ]:
knn(nona_list, nona_categorical, 1)

Fitting 5 folds for each of 136 candidates, totalling 680 fits
[CV] END  max_error: (test=10.889) mean_absolute_error: (test=4.550) root_mean_squared_error: (test=5.407) total time=   0.0s
[CV] END  max_error: (test=9.722) mean_absolute_error: (test=3.952) root_mean_squared_error: (test=4.957) total time=   0.0s
[CV] END  max_error: (test=18.167) mean_absolute_error: (test=6.360) root_mean_squared_error: (test=7.909) total time=   0.0s
[CV] END  max_error: (test=18.889) mean_absolute_error: (test=5.024) root_mean_squared_error: (test=6.940) total time=   0.0s
[CV] END  max_error: (test=29.833) mean_absolute_error: (test=4.544) root_mean_squared_error: (test=7.992) total time=   0.0s
{'knn__n_neighbors': np.int64(18), 'knn__p': 2, 'knn__weights': 'uniform'}
Train results
mae = 4.8861, root_mse = 6.6410, max_error = 17.5000

Test results
mae: 4.2393, root_mse: 5.4438, max_error: 12.1667


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [ ]:
knn(nona_list, nona_categorical, 2)

Fitting 5 folds for each of 136 candidates, totalling 680 fits
[CV] END  max_error: (test=18.083) mean_absolute_error: (test=4.150) root_mean_squared_error: (test=5.746) total time=   0.0s
[CV] END  max_error: (test=20.667) mean_absolute_error: (test=4.804) root_mean_squared_error: (test=6.606) total time=   0.0s
[CV] END  max_error: (test=28.042) mean_absolute_error: (test=5.326) root_mean_squared_error: (test=8.756) total time=   0.0s
[CV] END  max_error: (test=16.333) mean_absolute_error: (test=3.971) root_mean_squared_error: (test=5.345) total time=   0.0s
[CV] END  max_error: (test=18.750) mean_absolute_error: (test=4.980) root_mean_squared_error: (test=7.047) total time=   0.0s
{'knn__n_neighbors': np.int64(24), 'knn__p': 2, 'knn__weights': 'uniform'}
Train results
mae = 4.6463, root_mse = 6.7001, max_error = 20.3750

Test results
mae: 9.3277, root_mse: 11.6739, max_error: 19.4167


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


### Decision Tree

In [ ]:
def dtr(data_list, idx):
    X_train_ = data_list[idx][0]
    X_test_ = data_list[idx][1]
    y_train_ = data_list[idx][2]
    y_test_ = data_list[idx][3]
    parameters = {
        'min_samples_split' : [2, 3, 4, 5, 7, 10, 12, 15],
        'max_depth' : [*range(2, 15, 1), None]
    }
    model = DecisionTreeRegressor()
    best = GridSearchCV(
        estimator=model, param_grid=parameters,
        cv=kfold, verbose=1, refit=True,
        scoring='neg_root_mean_squared_error'
    )
    best.fit(X_train_, y_train_)
    best_params[idx]['DTR'].append(best.best_params_)
    model = DecisionTreeRegressor(
        max_depth=best_params[idx]['DTR'][-1]['max_depth'],
        min_samples_split=best_params[idx]['DTR'][-1]['min_samples_split'],
    )
    results = cross_validate(
        estimator=model, X=X_train_, y=y_train_,
        cv=kfold, verbose=3, scoring=scoring
    )

    model = best.best_estimator_
    y_pred = model.predict(X_test_)
    rmse = root_mean_squared_error(y_test_, y_pred)
    mae = mean_absolute_error(y_test_, y_pred)
    me = max_error(y_test_, y_pred)
    metrics = {'mae': mae, 'root_mse': rmse, 'max_error': me}
    best_metrics[idx]['DTR'].append(metrics)

    print(best.best_params_)
    print('\n''Train results')
    print_results(results)
    print('\n''Test results')
    print(f'mae: {mae:0.4f}, root_mse: {rmse:0.4f}, max_error: {me:0.4f}')

#### Decision Tree with NaN

In [ ]:
dtr(na_list, 1)

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV] END  max_error: (test=9.000) mean_absolute_error: (test=4.115) root_mean_squared_error: (test=4.660) total time=   0.0s
[CV] END  max_error: (test=15.000) mean_absolute_error: (test=4.197) root_mean_squared_error: (test=5.531) total time=   0.0s
[CV] END  max_error: (test=17.526) mean_absolute_error: (test=6.577) root_mean_squared_error: (test=8.139) total time=   0.0s
[CV] END  max_error: (test=18.591) mean_absolute_error: (test=6.425) root_mean_squared_error: (test=8.314) total time=   0.0s
[CV] END  max_error: (test=31.417) mean_absolute_error: (test=6.322) root_mean_squared_error: (test=9.390) total time=   0.0s
{'max_depth': 4, 'min_samples_split': 7}

Train results
mae = 5.5271, root_mse = 7.2068, max_error = 18.3068

Test results
mae: 4.4762, root_mse: 5.0973, max_error: 9.1200


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [ ]:
dtr(na_list, 2)

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV] END  max_error: (test=21.000) mean_absolute_error: (test=6.163) root_mean_squared_error: (test=8.797) total time=   0.0s
[CV] END  max_error: (test=15.042) mean_absolute_error: (test=5.729) root_mean_squared_error: (test=7.208) total time=   0.0s
[CV] END  max_error: (test=22.182) mean_absolute_error: (test=5.711) root_mean_squared_error: (test=7.909) total time=   0.0s
[CV] END  max_error: (test=46.000) mean_absolute_error: (test=6.850) root_mean_squared_error: (test=12.462) total time=   0.0s
[CV] END  max_error: (test=25.980) mean_absolute_error: (test=5.874) root_mean_squared_error: (test=8.688) total time=   0.0s
{'max_depth': 2, 'min_samples_split': 12}

Train results
mae = 6.0654, root_mse = 9.0128, max_error = 26.0407

Test results
mae: 6.2095, root_mse: 8.7759, max_error: 21.8293


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


#### Decision Tree without NaN

In [ ]:
dtr(nona_list, 1)

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV] END  max_error: (test=9.562) mean_absolute_error: (test=3.391) root_mean_squared_error: (test=4.322) total time=   0.0s
[CV] END  max_error: (test=15.000) mean_absolute_error: (test=4.471) root_mean_squared_error: (test=5.804) total time=   0.0s
[CV] END  max_error: (test=15.667) mean_absolute_error: (test=6.478) root_mean_squared_error: (test=7.634) total time=   0.0s
[CV] END  max_error: (test=18.125) mean_absolute_error: (test=5.652) root_mean_squared_error: (test=7.445) total time=   0.0s
[CV] END  max_error: (test=17.000) mean_absolute_error: (test=5.393) root_mean_squared_error: (test=6.902) total time=   0.0s
{'max_depth': 3, 'min_samples_split': 2}

Train results
mae = 5.0769, root_mse = 6.4213, max_error = 15.0708

Test results
mae: 4.2266, root_mse: 5.1147, max_error: 10.0870


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [ ]:
dtr(nona_list, 2)

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV] END  max_error: (test=17.091) mean_absolute_error: (test=4.245) root_mean_squared_error: (test=5.677) total time=   0.0s
[CV] END  max_error: (test=46.000) mean_absolute_error: (test=6.671) root_mean_squared_error: (test=12.298) total time=   0.0s
[CV] END  max_error: (test=46.000) mean_absolute_error: (test=6.651) root_mean_squared_error: (test=12.551) total time=   0.0s
[CV] END  max_error: (test=16.050) mean_absolute_error: (test=4.087) root_mean_squared_error: (test=5.400) total time=   0.0s
[CV] END  max_error: (test=25.000) mean_absolute_error: (test=6.213) root_mean_squared_error: (test=9.102) total time=   0.0s
{'max_depth': 3, 'min_samples_split': 2}

Train results
mae = 5.5735, root_mse = 9.0055, max_error = 30.0282

Test results
mae: 10.0669, root_mse: 14.0613, max_error: 46.0000


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
